<a href="https://colab.research.google.com/github/mcucii/ML-projects/blob/main/Movie%20Recommender%20Systems/Movie_Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os

# https://www.kaggle.com/code/rounakbanik/movie-recommender-systems/notebook

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
movie_dir = '/content/drive/MyDrive/ML_projects/Movie Recommender Systems/movie_dataset'

In [148]:
os.listdir(movie_dir)

['credits.csv',
 'links_small.csv',
 'keywords.csv',
 'movies_metadata.csv',
 'links.csv',
 'ratings.csv',
 'ratings_small.csv']



### SIMPLE RECOMMENDER
- generalized recommendations based on popularity
- not customized



In [149]:
md = pd.read_csv(os.path.join(movie_dir, 'movies_metadata.csv'))

<ipython-input-149-711edf8feac2>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv(os.path.join(movie_dir, 'movies_metadata.csv'))


In [150]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [151]:
from ast import literal_eval
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

In [152]:
md['genres'].head()

,genres
0,"[Animation, Comedy, Family]"
1,"[Adventure, Fantasy, Family]"
2,"[Romance, Comedy]"
3,"[Comedy, Drama, Romance]"
4,[Comedy]



*IMDB formula for calculating top movies:*

$ WR = \frac{v}{m+v} * R + \frac{m}{v+m} * C$

- R = average rating for the movie (mean)
- C = the mean vote across the whole report
- v = number of votes for the movie
- m = min. votes required to be listed on top 250 -> more votes than 95% of the movies




In [153]:
# taking out nulls
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

np.float64(5.244896612406511)

In [154]:
m = vote_counts.quantile(0.95)   # 95 percentile
m

np.float64(434.0)

In [155]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x : str(x).split('-')[0] if x != np.nan else np.nan)

In [156]:
qualified = md[(md['vote_count'] != np.nan) & (md['vote_count'] >= m) & (md['vote_average'] != np.nan)][['title', 'year', 'vote_count', 'vote_average', 'genres', 'popularity']]
qualified['vote_count']= qualified['vote_count'].astype('int')
qualified['vote_average']= qualified['vote_average'].astype('int')


In [157]:
qualified.shape

(2274, 6)

In [158]:
def imdb_rating(x):
  v = x['vote_count']
  R = x['vote_average']
  return R*(v/(v+m)) + C*(m/(v+m))

In [159]:
qualified[qualified['title'] == 'Fight Club']

,title,year,vote_count,vote_average,genres,popularity
2843,Fight Club,1999,9678,8,[Drama],63.869599


In [160]:
qualified['imdb_rating'] = qualified.apply(imdb_rating, axis=1)

In [161]:
qualified[qualified['title'] == 'Fight Club']

,title,year,vote_count,vote_average,genres,popularity,imdb_rating
2843,Fight Club,1999,9678,8,[Drama],63.869599,7.881753


In [162]:
qualified = qualified.sort_values('imdb_rating', ascending=False).head(250)

In [163]:
# TOP MOVIES:
qualified.head(15)

,title,year,vote_count,vote_average,genres,popularity,imdb_rating
15480,Inception,2010,14075,8,"[Action, Thriller, Science Fiction, Mystery, A...",29.108149,7.917588
12481,The Dark Knight,2008,12269,8,"[Drama, Action, Crime, Thriller]",123.167259,7.905871
22879,Interstellar,2014,11187,8,"[Adventure, Drama, Science Fiction]",32.213481,7.897107
2843,Fight Club,1999,9678,8,[Drama],63.869599,7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,"[Adventure, Fantasy, Action]",32.070725,7.871787
292,Pulp Fiction,1994,8670,8,"[Thriller, Crime]",140.950236,7.868660
314,The Shawshank Redemption,1994,8358,8,"[Drama, Crime]",51.645403,7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,"[Adventure, Fantasy, Action]",29.324358,7.861927
351,Forrest Gump,1994,8147,8,"[Comedy, Drama, Romance]",48.307194,7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,"[Adventure, Fantasy, Action]",29.423537,7.851924


- Now, let's build charts for each genre

In [164]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name='genre'

In [165]:
s.head()

,genre
0,Animation
0,Comedy
0,Family
1,Adventure
1,Fantasy


In [166]:
g_md = md.drop('genres', axis=1).join(s)
#g_md.head()

In [183]:
def chart_by_genre(genre, percentile=0.85):
  df = g_md[g_md['genre'] == genre]
  vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
  vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
  C = vote_averages.mean()
  m = vote_counts.quantile(percentile)

  qualified = df[(df['vote_average'].notnull()) & (df['vote_count'].notnull()) & (df['vote_count']>=m)][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')

  qualified['rating'] = qualified.apply(lambda x : ((x['vote_count']/(x['vote_count'] + m))*x['vote_average'] + (m/(x['vote_count']+m))*C), axis=1)
  qualified = qualified.sort_values('rating', ascending=False).head(250)

  return qualified

In [185]:
top_romance_movies = chart_by_genre('Romance')
top_romance_movies.head(10)

,title,year,vote_count,vote_average,popularity,rating
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166
